In [57]:
#import needed packages
import os
import sys
import requests
import json
import pandas as pd
import time
import datetime

In [2]:
#add upper directories to path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [41]:
api_host='lihcode-lufthansa-open-new-v1.p.rapidapi.com'
api_key='74e0ebdf6cmsh0245562f7e4e7efp1bece0jsn07dde692e3a9'
origin="MUC"
destination="HEL"
fromdatetime="2019-08-22"
api_call='https://lihcode-lufthansa-open-new-v1.p.rapidapi.com/v1/operations/schedules/'+origin+'/'+destination+'/'+fromdatetime
headers = {
    "X-RapidAPI-Host": api_host,
    "X-RapidAPI-Key": api_key
    }

In [42]:
r=requests.get(api_call, headers=headers)

In [43]:
print(r.json())

{'ScheduleResource': {'Schedule': [{'TotalJourney': {'Duration': 'PT2H30M'}, 'Flight': {'Departure': {'AirportCode': 'MUC', 'ScheduledTimeLocal': {'DateTime': '2019-08-22T08:25'}, 'Terminal': {'Name': 2}}, 'Arrival': {'AirportCode': 'HEL', 'ScheduledTimeLocal': {'DateTime': '2019-08-22T11:55'}, 'Terminal': {'Name': 1}}, 'MarketingCarrier': {'AirlineID': 'LH', 'FlightNumber': 2462}, 'Equipment': {'AircraftCode': 321}, 'Details': {'Stops': {'StopQuantity': 0}, 'DaysOfOperation': 34567, 'DatePeriod': {'Effective': '2019-08-22', 'Expiration': '2019-08-29'}}}}, {'TotalJourney': {'Duration': 'PT2H30M'}, 'Flight': {'Departure': {'AirportCode': 'MUC', 'ScheduledTimeLocal': {'DateTime': '2019-08-22T11:05'}, 'Terminal': {'Name': 2}}, 'Arrival': {'AirportCode': 'HEL', 'ScheduledTimeLocal': {'DateTime': '2019-08-22T14:35'}, 'Terminal': {'Name': 1}}, 'MarketingCarrier': {'AirlineID': 'LH', 'FlightNumber': 2460}, 'Equipment': {'AircraftCode': 320}, 'Details': {'Stops': {'StopQuantity': 0}, 'DaysOfOp

In [44]:
body = json.loads(r.text)

In [47]:
body.keys()

dict_keys(['ScheduleResource'])

In [52]:
type(body['ScheduleResource']['Schedule'][3]['Flight'])

dict

In [60]:
#####INITIAL TESTS
flights=[]
flight={}
flight['Departure_airport']='MUC-sky'
flight['Destination_airport']='HEL-sky'
flight['Date']=fromdatetime
tempflight=[]

for i in range(len(body['ScheduleResource']['Schedule'])):
    if type(body['ScheduleResource']['Schedule'][i]['Flight']) is dict:
        tempflights={}
        tempflights['Departure']=body['ScheduleResource']['Schedule'][i]['Flight']['Departure']['ScheduledTimeLocal']['DateTime']
        tempflights['Arrival']=body['ScheduleResource']['Schedule'][i]['Flight']['Arrival']['ScheduledTimeLocal']['DateTime']
        date_time_obj_dep = datetime.datetime.strptime(tempflights['Departure'], '%Y-%m-%dT%H:%M')
        date_time_obj_arr = datetime.datetime.strptime(tempflights['Arrival'], '%Y-%m-%dT%H:%M')
        tempflights['Duration']=int((date_time_obj_arr-date_time_obj_dep).total_seconds()/60)
        tempflight.append(tempflights)
flight['Flights']=tempflight
flights.append(flight)

In [54]:
body['ScheduleResource']['Schedule'][3]['Flight']

{'Departure': {'AirportCode': 'MUC',
  'ScheduledTimeLocal': {'DateTime': '2019-08-22T20:00'},
  'Terminal': {'Name': 2}},
 'Arrival': {'AirportCode': 'HEL',
  'ScheduledTimeLocal': {'DateTime': '2019-08-22T23:30'},
  'Terminal': {'Name': 1}},
 'MarketingCarrier': {'AirlineID': 'LH', 'FlightNumber': 2466},
 'Equipment': {'AircraftCode': 320},
 'Details': {'Stops': {'StopQuantity': 0},
  'DaysOfOperation': 124567,
  'DatePeriod': {'Effective': '2019-08-22', 'Expiration': '2019-08-27'}}}

In [61]:
flights

[{'Departure_airport': 'MUC-sky',
  'Destination_airport': 'HEL-sky',
  'Date': '2019-08-22',
  'Flights': [{'Departure': '2019-08-22T08:25',
    'Arrival': '2019-08-22T11:55',
    'Duration': 210},
   {'Departure': '2019-08-22T11:05',
    'Arrival': '2019-08-22T14:35',
    'Duration': 210},
   {'Departure': '2019-08-22T15:05',
    'Arrival': '2019-08-22T18:35',
    'Duration': 210},
   {'Departure': '2019-08-22T20:00',
    'Arrival': '2019-08-22T23:30',
    'Duration': 210}]}]

In [66]:
######################### MUNICH TO HELSINKI ################
all_flights=[]
munich_flights=[]
api_host='lihcode-lufthansa-open-new-v1.p.rapidapi.com'
api_key='74e0ebdf6cmsh0245562f7e4e7efp1bece0jsn07dde692e3a9'
origin="MUC"
destination="HEL"
dates=pd.date_range("2019-07-21","2019-08-31")
dates=dates.astype("str")
for date in dates:
    api_call='https://lihcode-lufthansa-open-new-v1.p.rapidapi.com/v1/operations/schedules/'+origin+'/'+destination+'/'+date
    headers = {
        "X-RapidAPI-Host": api_host,
        "X-RapidAPI-Key": api_key
        }
    r=requests.get(api_call, headers=headers)
    body = json.loads(r.text)
    flight={}
    flight['Departure_airport']='MUC-sky'
    flight['Destination_airport']='HEL-sky'
    flight['Date']=date
    tempflight=[]

    for i in range(len(body['ScheduleResource']['Schedule'])):
        if type(body['ScheduleResource']['Schedule'][i]['Flight']) is dict:
            tempflights={}
            tempflights['Departure']=body['ScheduleResource']['Schedule'][i]['Flight']['Departure']['ScheduledTimeLocal']['DateTime']
            tempflights['Arrival']=body['ScheduleResource']['Schedule'][i]['Flight']['Arrival']['ScheduledTimeLocal']['DateTime']
            date_time_obj_dep = datetime.datetime.strptime(tempflights['Departure'], '%Y-%m-%dT%H:%M')
            date_time_obj_arr = datetime.datetime.strptime(tempflights['Arrival'], '%Y-%m-%dT%H:%M')
            tempflights['Duration']=int((date_time_obj_arr-date_time_obj_dep).total_seconds()/60)
            tempflight.append(tempflights)
    flight['Flights']=tempflight
    munich_flights.append(flight)
all_flights.extend(munich_flights)

In [67]:
all_flights

[{'Departure_airport': 'MUC-sky',
  'Destination_airport': 'HEL-sky',
  'Date': '2019-07-21',
  'Flights': [{'Departure': '2019-07-21T08:25',
    'Arrival': '2019-07-21T11:55',
    'Duration': 210},
   {'Departure': '2019-07-21T11:05',
    'Arrival': '2019-07-21T14:35',
    'Duration': 210},
   {'Departure': '2019-07-21T15:05',
    'Arrival': '2019-07-21T18:35',
    'Duration': 210},
   {'Departure': '2019-07-21T20:00',
    'Arrival': '2019-07-21T23:30',
    'Duration': 210}]},
 {'Departure_airport': 'MUC-sky',
  'Destination_airport': 'HEL-sky',
  'Date': '2019-07-22',
  'Flights': [{'Departure': '2019-07-22T08:25',
    'Arrival': '2019-07-22T11:55',
    'Duration': 210},
   {'Departure': '2019-07-22T11:05',
    'Arrival': '2019-07-22T14:35',
    'Duration': 210},
   {'Departure': '2019-07-22T15:05',
    'Arrival': '2019-07-22T18:35',
    'Duration': 210},
   {'Departure': '2019-07-22T20:00',
    'Arrival': '2019-07-22T23:30',
    'Duration': 210}]},
 {'Departure_airport': 'MUC-sky',


In [69]:
######################### HELSINKI TO MUNICH ################
hel_munich_flights=[]
api_host='lihcode-lufthansa-open-new-v1.p.rapidapi.com'
api_key='74e0ebdf6cmsh0245562f7e4e7efp1bece0jsn07dde692e3a9'
origin="HEL"
destination="MUC"
dates=pd.date_range("2019-07-21","2019-08-31")
dates=dates.astype("str")
for date in dates:
    api_call='https://lihcode-lufthansa-open-new-v1.p.rapidapi.com/v1/operations/schedules/'+origin+'/'+destination+'/'+date
    headers = {
        "X-RapidAPI-Host": api_host,
        "X-RapidAPI-Key": api_key
        }
    r=requests.get(api_call, headers=headers)
    body = json.loads(r.text)
    flight={}
    flight['Departure_airport']='HEL-sky'
    flight['Destination_airport']='MUC-sky'
    flight['Date']=date
    tempflight=[]

    for i in range(len(body['ScheduleResource']['Schedule'])):
        if type(body['ScheduleResource']['Schedule'][i]['Flight']) is dict:
            tempflights={}
            tempflights['Departure']=body['ScheduleResource']['Schedule'][i]['Flight']['Departure']['ScheduledTimeLocal']['DateTime']
            tempflights['Arrival']=body['ScheduleResource']['Schedule'][i]['Flight']['Arrival']['ScheduledTimeLocal']['DateTime']
            date_time_obj_dep = datetime.datetime.strptime(tempflights['Departure'], '%Y-%m-%dT%H:%M')
            date_time_obj_arr = datetime.datetime.strptime(tempflights['Arrival'], '%Y-%m-%dT%H:%M')
            tempflights['Duration']=int((date_time_obj_arr-date_time_obj_dep).total_seconds()/60)
            tempflight.append(tempflights)
    flight['Flights']=tempflight
    hel_munich_flights.append(flight)
all_flights.extend(hel_munich_flights)

In [70]:
######################### AMSTERDAM TO MUNICH ################
ams_munich_flights=[]
api_host='lihcode-lufthansa-open-new-v1.p.rapidapi.com'
api_key='74e0ebdf6cmsh0245562f7e4e7efp1bece0jsn07dde692e3a9'
origin="AMS"
destination="MUC"
dates=pd.date_range("2019-07-21","2019-08-31")
dates=dates.astype("str")
for date in dates:
    api_call='https://lihcode-lufthansa-open-new-v1.p.rapidapi.com/v1/operations/schedules/'+origin+'/'+destination+'/'+date
    headers = {
        "X-RapidAPI-Host": api_host,
        "X-RapidAPI-Key": api_key
        }
    r=requests.get(api_call, headers=headers)
    body = json.loads(r.text)
    flight={}
    flight['Departure_airport']='AMS-sky'
    flight['Destination_airport']='MUC-sky'
    flight['Date']=date
    tempflight=[]

    for i in range(len(body['ScheduleResource']['Schedule'])):
        if type(body['ScheduleResource']['Schedule'][i]['Flight']) is dict:
            tempflights={}
            tempflights['Departure']=body['ScheduleResource']['Schedule'][i]['Flight']['Departure']['ScheduledTimeLocal']['DateTime']
            tempflights['Arrival']=body['ScheduleResource']['Schedule'][i]['Flight']['Arrival']['ScheduledTimeLocal']['DateTime']
            date_time_obj_dep = datetime.datetime.strptime(tempflights['Departure'], '%Y-%m-%dT%H:%M')
            date_time_obj_arr = datetime.datetime.strptime(tempflights['Arrival'], '%Y-%m-%dT%H:%M')
            tempflights['Duration']=int((date_time_obj_arr-date_time_obj_dep).total_seconds()/60)
            tempflight.append(tempflights)
    flight['Flights']=tempflight
    ams_munich_flights.append(flight)
all_flights.extend(ams_munich_flights)

In [71]:
#########################  MUNICH TO AMSTERDAM ################
munich_ams_flights=[]
api_host='lihcode-lufthansa-open-new-v1.p.rapidapi.com'
api_key='74e0ebdf6cmsh0245562f7e4e7efp1bece0jsn07dde692e3a9'
origin="AMS"
destination="MUC"
dates=pd.date_range("2019-07-21","2019-08-31")
dates=dates.astype("str")
for date in dates:
    api_call='https://lihcode-lufthansa-open-new-v1.p.rapidapi.com/v1/operations/schedules/'+origin+'/'+destination+'/'+date
    headers = {
        "X-RapidAPI-Host": api_host,
        "X-RapidAPI-Key": api_key
        }
    r=requests.get(api_call, headers=headers)
    body = json.loads(r.text)
    flight={}
    flight['Departure_airport']='MUC-sky'
    flight['Destination_airport']='AMS-sky'
    flight['Date']=date
    tempflight=[]

    for i in range(len(body['ScheduleResource']['Schedule'])):
        if type(body['ScheduleResource']['Schedule'][i]['Flight']) is dict:
            tempflights={}
            tempflights['Departure']=body['ScheduleResource']['Schedule'][i]['Flight']['Departure']['ScheduledTimeLocal']['DateTime']
            tempflights['Arrival']=body['ScheduleResource']['Schedule'][i]['Flight']['Arrival']['ScheduledTimeLocal']['DateTime']
            date_time_obj_dep = datetime.datetime.strptime(tempflights['Departure'], '%Y-%m-%dT%H:%M')
            date_time_obj_arr = datetime.datetime.strptime(tempflights['Arrival'], '%Y-%m-%dT%H:%M')
            tempflights['Duration']=int((date_time_obj_arr-date_time_obj_dep).total_seconds()/60)
            tempflight.append(tempflights)
    flight['Flights']=tempflight
    munich_ams_flights.append(flight)
all_flights.extend(munich_ams_flights)

In [76]:
all_flights

[{'Departure_airport': 'MUC-sky',
  'Destination_airport': 'HEL-sky',
  'Date': '2019-07-21',
  'Flights': [{'Departure': '2019-07-21T08:25',
    'Arrival': '2019-07-21T11:55',
    'Duration': 210},
   {'Departure': '2019-07-21T11:05',
    'Arrival': '2019-07-21T14:35',
    'Duration': 210},
   {'Departure': '2019-07-21T15:05',
    'Arrival': '2019-07-21T18:35',
    'Duration': 210},
   {'Departure': '2019-07-21T20:00',
    'Arrival': '2019-07-21T23:30',
    'Duration': 210}]},
 {'Departure_airport': 'MUC-sky',
  'Destination_airport': 'HEL-sky',
  'Date': '2019-07-22',
  'Flights': [{'Departure': '2019-07-22T08:25',
    'Arrival': '2019-07-22T11:55',
    'Duration': 210},
   {'Departure': '2019-07-22T11:05',
    'Arrival': '2019-07-22T14:35',
    'Duration': 210},
   {'Departure': '2019-07-22T15:05',
    'Arrival': '2019-07-22T18:35',
    'Duration': 210},
   {'Departure': '2019-07-22T20:00',
    'Arrival': '2019-07-22T23:30',
    'Duration': 210}]},
 {'Departure_airport': 'MUC-sky',


In [77]:
with open('flights2.json', 'w') as fout:
    json.dump(all_flights, fout)